# Variational AutoEncoder (MLP) Tensorflow
Train a variational autoencoder with MNIST dataset

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/kvfrans/variational-autoencoder
* https://github.com/int8/VAE_tensorflow
* http://int8.io/variational-autoencoder-in-tensorflow/
* http://blog.fastforwardlabs.com/2016/08/22/under-the-hood-of-the-variational-autoencoder-in.html
* http://blog.fastforwardlabs.com/2016/08/12/introducing-variational-autoencoders-in-prose-and.html
* https://blog.keras.io/building-autoencoders-in-keras.html
* https://jaan.io/what-is-variational-autoencoder-vae-tutorial/
* https://arxiv.org/pdf/1606.05908.pdf
* https://arxiv.org/pdf/1312.6114.pdf
* http://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/
* https://www.tensorflow.org/get_started/embedding_viz
* https://www.youtube.com/watch?v=eBbEDRsCmv4
* https://github.com/normanheckscher/mnist-tensorboard-embeddings
* http://projector.tensorflow.org/

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)

# Delete directory if exist
if os.path.exists('/tmp/vae_cnn'):    
    os.system("rm -rf /tmp/vae_cnn")

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
from models import VAE_CNN
model = VAE_CNN(latent_size = 20)
model_in = model.input
model_out = model.output
model_out_flat = model.output_flat
z_mean = model.z_mean
z_stddev = model.z_stddev

### Define loss

In [3]:
with tf.name_scope("VAE_LOSS"):
    # Binary cross entropy
    #generation_loss = -tf.reduce_sum(model_in * tf.log(1e-8 + model_out_flat) + (1-model_in) * tf.log(1e-8 + 1 - model_out_flat),1)
    
    # L2 Loss
    generation_loss = tf.nn.l2_loss(model_in-model_out_flat)
    
    # KL Loss
    latent_loss = 0.5 * tf.reduce_sum(tf.square(z_mean) + tf.square(z_stddev) - tf.log(tf.square(z_stddev)) - 1,1)
    
    # Merge the losses
    loss = tf.reduce_mean(generation_loss + latent_loss)

# Solver configuration
with tf.name_scope("Solver"):
    train_step = tf.train.AdamOptimizer(0.00001).minimize(loss)

### Build Graph

In [4]:
init = tf.global_variables_initializer()

# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

#sess = tf.Session()
sess.run(init)

### Add some tensors to observe on Tensorboard

In [5]:
tf.summary.image("input_image", model.image_in, 4)
tf.summary.image("output_image", model_out, 4)
tf.summary.scalar("loss", loss)

merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("/tmp/vae_cnn/1")
writer.add_graph(sess.graph)

# Create saver object
saver = tf.train.Saver()

### Train

In [6]:
num_epoch = 100
batch_size = 100
for epoch in range(num_epoch):
    for i in range(int(mnist.train.num_examples / batch_size)):
        # Get batch of 50 images
        batch = mnist.train.next_batch(50)

        # Dump summary
        if i % 5000 == 0:            
            # Other summaries
            s = sess.run(merged_summary, feed_dict={model_in:batch[0]})
            writer.add_summary(s,i) 
            
            # Save embedding (for PCA, TSNE)
            sess.run(model.assignment, feed_dict={model_in: batch[0]})
            saver.save(sess, os.path.join("/tmp/vae_cnn/", "model.ckpt"), i)


        # Train actually here (Also get loss value)    
        _, val_loss = sess.run((train_step, loss), feed_dict={model_in:batch[0]})

        #if i % 5000:
            #print('Generation loss: %d, latent loss: %d' % (np.mean(gen_loss), np.mean(lat_loss)))
    print('Epoch: %d/%d loss:%d' % (epoch, num_epoch, val_loss))

Epoch: 0/100 loss:3816
Epoch: 1/100 loss:2096
Epoch: 2/100 loss:1827
Epoch: 3/100 loss:1655
Epoch: 4/100 loss:1572
Epoch: 5/100 loss:1508
Epoch: 6/100 loss:1397
Epoch: 7/100 loss:1415
Epoch: 8/100 loss:1296
Epoch: 9/100 loss:1266
Epoch: 10/100 loss:1358
Epoch: 11/100 loss:1262
Epoch: 12/100 loss:1254
Epoch: 13/100 loss:1205
Epoch: 14/100 loss:1068
Epoch: 15/100 loss:1018
Epoch: 16/100 loss:954
Epoch: 17/100 loss:880
Epoch: 18/100 loss:823
Epoch: 19/100 loss:812
Epoch: 20/100 loss:734
Epoch: 21/100 loss:750
Epoch: 22/100 loss:645
Epoch: 23/100 loss:665
Epoch: 24/100 loss:648
Epoch: 25/100 loss:650
Epoch: 26/100 loss:648
Epoch: 27/100 loss:600
Epoch: 28/100 loss:592
Epoch: 29/100 loss:586
Epoch: 30/100 loss:517
Epoch: 31/100 loss:569
Epoch: 32/100 loss:565
Epoch: 33/100 loss:499
Epoch: 34/100 loss:534
Epoch: 35/100 loss:518
Epoch: 36/100 loss:520
Epoch: 37/100 loss:461
Epoch: 38/100 loss:491
Epoch: 39/100 loss:475
Epoch: 40/100 loss:521
Epoch: 41/100 loss:451
Epoch: 42/100 loss:456
Epoch